In [443]:
#importing the required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [444]:
data=pd.read_csv('cpdata.csv')
print(data.head(1))

   temperature   humidity    rainfall label
0    20.879744  82.002744  202.935536  rice


In [445]:
#Creating dummy variable for target i.e label
label= pd.get_dummies(data.label).iloc[: , 1:]
data= pd.concat([data,label],axis=1)
data.drop('label', axis=1,inplace=True)
print('The data present in one row of the dataset is')
print(data.head(2))
column_headers = list(data.columns.values)
print("The Column Header :", column_headers)
train=data.iloc[:, 0:3].values
test=data.iloc[: ,3:].values

The data present in one row of the dataset is
   temperature   humidity    rainfall  Black gram  Chickpea  Coconut  Coffee  \
0    20.879744  82.002744  202.935536           0         0        0       0   
1    21.770462  80.319644  226.655537           0         0        0       0   

   Cotton  Ground Nut  Jute  ...  maize  mango  millet  muskmelon  orange  \
0       0           0     0  ...      0      0       0          0       0   
1       0           0     0  ...      0      0       0          0       0   

   papaya  pomegranate  rice  watermelon  wheat  
0       0            0     1           0      0  
1       0            0     1           0      0  

[2 rows x 33 columns]
The Column Header : ['temperature', 'humidity', 'rainfall', 'Black gram', 'Chickpea', 'Coconut', 'Coffee', 'Cotton', 'Ground Nut', 'Jute', 'Kidney Beans', 'Lentil', 'Moth Beans', 'Mung Bean', 'Peas', 'Pigeon Peas', 'Rubber', 'Sugarcane', 'Tea', 'Tobacco', 'apple', 'banana', 'grapes', 'maize', 'mango', 'mill

In [446]:
#Dividing the data into training and test set
X_train,X_test,y_train,y_test=train_test_split(train,test,test_size=0.1)
print(X_train.shape)
print(X_test[0])
print(y_train.shape)
print(y_test[0])

(2790, 3)
[19.0781471  69.02298571 80.72515943]
(2790, 30)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]


In [447]:
def init_var():
    w1 = np.random.rand(30, 3) - 0.5
    b1 = np.random.rand(30, 1) - 0.5
    w2 = np.random.rand(30, 30) - 0.5
    b2 = np.random.rand(30, 1) - 0.5
    return w1, b1, w2, b2

In [448]:
def ReLU(z):
    return np.maximum(z, 0)

def Softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def derv_ReLU(z):
    return z > 0

In [449]:
# arr is a 2d numpy array, assign the maximum value in each row to 1 and the rest to 0
def max_to_one(arr, m, n):
    for i in range(0, m):
        d = np.argmax(arr[i])
        for j in range(0, n):
            if(d == j):
                arr[i][j] = 1
            else:
                arr[i][j] = 0
    return arr

In [450]:
def forward_prop(w1, b1, w2, b2, x):
    z1 = w1.dot(x) + b1
    a1 = ReLU(z1)
    z2 = w2.dot(a1) + b2
    a2 = Softmax(z2)
    a2 = a2.T
    #find the number of rows in a2
    m = a2.shape[0]
    a2 = max_to_one(a2, m, 30)
    # print(a2)
    # print(a2.shape)
    return z1, a1, z2, a2

def back_prop(w1, b1, w2, b2, x, y, z1, a1, z2, a2):
    m = y.size
    dz2 = (a2 - y).T
    dw2 = 1 / m * (dz2.dot(a1.T))
    db2 = 1 / m * np.sum(dz2)
    dz1 = w2.T.dot(dz2) * derv_ReLU(z1)
    dw1 = 1 / m * (dz1.dot(x.T))
    db1 = 1 / m * np.sum(dz1)
    return dw1, db1, dw2, db2

def update_var(w1, b1, w2, b2, dw1, db1, dw2, db2, learning_rate):
    w1 = w1 - learning_rate * dw1
    b1 = b1 - learning_rate * db1
    w2 = w2 - learning_rate * dw2
    b2 = b2 - learning_rate * db2
    return w1, b1, w2, b2

In [451]:
def accuracy(y_pred, y):
    return np.mean(y_pred == y)

def grad_descent(x, y, alpha, w1, b1, w2, b2):
    z1, a1, z2, a2 = forward_prop(w1, b1, w2, b2, x)
    dw1, db1, dw2, db2 = back_prop(w1, b1, w2, b2, x, y, z1, a1, z2, a2)
    w1, b1, w2, b2 = update_var(w1, b1, w2, b2, dw1, db1, dw2, db2, alpha)
    return w1, b1, w2, b2, a2

In [452]:
from sklearn.utils import shuffle
batch_size = 32
n_example = 2790
batches = int(n_example/batch_size)
epochs = 100
w1, b1, w2, b2 = init_var()
acc = np.empty(0)
for epoch in range(epochs):
    i = shuffle(range(n_example))
    for b in range(batches):
        x_batch = X_train[i[b*batch_size:(b+1)*batch_size]]
        y_batch = y_train[i[b*batch_size:(b+1)*batch_size]]
        w1, b1, w2, b2, a2 = grad_descent(x_batch.T, y_batch, 0.1, w1, b1, w2, b2)
        acc = np.append(acc, accuracy(a2, y_batch))
    print("Iteration: ", epoch)
    print("Accuracy: ", np.mean(acc))

Iteration:  0
Accuracy:  0.9377993295019156
Iteration:  1
Accuracy:  0.9384339080459769
Iteration:  2
Accuracy:  0.938661398467433
Iteration:  3
Accuracy:  0.9387811302681992
Iteration:  4
Accuracy:  0.9388433908045978
Iteration:  5
Accuracy:  0.9388908844189017
Iteration:  6
Accuracy:  0.9389265188834154
Iteration:  7
Accuracy:  0.9389442648467432
Iteration:  8
Accuracy:  0.9389660493827162
Iteration:  9
Accuracy:  0.9389834770114943
Iteration:  10
Accuracy:  0.9390042668059909
Iteration:  11
Accuracy:  0.9390176005747126
Iteration:  12
Accuracy:  0.9390270409666962
Iteration:  13
Accuracy:  0.9390368431855501
Iteration:  14
Accuracy:  0.9390437420178798
Iteration:  15
Accuracy:  0.9390497784961686
Iteration:  16
Accuracy:  0.9390544004958304
Iteration:  17
Accuracy:  0.939053852703278
Iteration:  18
Accuracy:  0.939059664246824
Iteration:  19
Accuracy:  0.9390636973180077
Iteration:  20
Accuracy:  0.9390662059843095
Iteration:  21
Accuracy:  0.9390717520027865
Iteration:  22
Accuracy

In [479]:
atemp, ah, rain = 27.98, 84.58, 136.72
l=[]
l.append(atemp)
l.append(ah)
l.append(rain)
#create a numpy arrray with atemp, ah and rain
predictcrop = np.array(l)
predictcrop = predictcrop.reshape(1, 3)
print(predictcrop)
print(predictcrop.shape)

[[ 27.98  84.58 136.72]]
(1, 3)


In [454]:
# Putting the names of crop in a single list
crops=['Black gram', 'Chickpea', 'Coconut', 'Coffee', 'Cotton', 'Ground Nut', 'Jute', 'Kidney Beans', 'Lentil', 'Moth Beans', 'Mung Bean', 'Peas', 'Pigeon Peas', 'Rubber', 'Sugarcane', 'Tea', 'Tobacco', 'apple', 'banana', 'grapes', 'maize', 'mango', 'millet', 'muskmelon', 'orange', 'papaya', 'pomegranate', 'rice', 'watermelon', 'wheat']
print(len(crops))

30


In [455]:
def pred(w1, b1, w2, b2, x):
    z1 = w1.dot(x) + b1
    a1 = ReLU(z1)
    z2 = w2.dot(a1) + b2
    a2 = Softmax(z2)
    a2 = a2.T
    #a2 = max_to_one(a2, 30, 30)
    return a2

In [484]:
# print(w1.shape)
# print(b1.shape)
# print(w2.shape)
# print(b2.shape)
# print(predictcrop.shape)
# x1 = w1
# x2 = b1
# x3 = w2
# x4 = b2
print(predictcrop)
a2 = pred(w1, b1, w2, b2, predictcrop.T)
print(a2.shape)
print(a2)
#find mean of each column of a2
#a2 = np.mean(a2, axis=0)
#find maximum in a2
d = np.argmax(a2[0])
#find the index of the second largest number in a2
d2 = np.argpartition(a2.flatten(), -2)[-2]
print("The crop recommended is ",crops[d])
print(d)
print("The second crop recommended is ",crops[d2])
print(d2)
#print(a2)

[[ 27.98  84.58 136.72]]
(1, 30)
[[0.03116448 0.03573726 0.04494595 0.041833   0.04912877 0.04363539
  0.03159376 0.0325121  0.05255235 0.02366188 0.05441477 0.03077614
  0.0223424  0.02114634 0.03213497 0.03711945 0.04737735 0.02358905
  0.04128414 0.0358893  0.02319309 0.0302635  0.02477444 0.02084793
  0.04309872 0.02692784 0.02110092 0.02244417 0.02993451 0.02457603]]
The crop recommended is  Mung Bean
10
The second crop recommended is  Lentil
8
